## DataSet Iterator

tf.Data API 目前支持下列的迭代器。

- 单次(one-shot -- make_one_hot_iterator)
- 可初始化 (initializable -- make_initializer_iterator)
- 可重新初始化(reinitializable)
- 可馈送(feedable)


In [4]:
import tensorflow.compat.v1 as tf

tf.disable_eager_execution()

#### 1. 单次

单次迭代器是最简单的迭代器形式，仅支持对数据集进行一次迭代，不需要显式初始化。单次迭代器可以处理基于队列的现有输入管道支持的几乎所有情况，但它们不支持参数化。以 Dataset.range() 为例

In [5]:
dataset = tf.data.Dataset.range(100)
iterator = tf.data.make_one_shot_iterator(dataset)
next_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(100):
        value = sess.run(next_element)
        assert i == value

#### 2. 可初始化

与 `tf.placehodler` 相结合。这需要我们先显示运行 `iterator.initializer` 操作，然后才能使用可初始化的迭代器。

In [10]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = tf.data.make_initializable_iterator(dataset)
next_element = iterator.get_next()

with tf.Session() as sess:
    # 需要我们先运行 iterator 的 initialiizer
    sess.run(iterator.initializer, {max_value: 10})
    for i in range(10):
        value = sess.run(next_element)
        assert i == value
        
# 通过初始化，我们可复用自定义数据集
with tf.Session() as sess:
    # 需要我们先运行 iterator 的 initialiizer
    sess.run(iterator.initializer, {max_value: 100})
    for i in range(10):
        value = sess.run(next_element)
        assert i == value

### 3. 可重新初始化

可重新初始化可以通过对多个不同 Dataset 对象进行初始化。如**训练集与验证集**相结合。虽然数据集不同，但是这些 Dataset 具有相同的类型的兼容形状。


In [20]:
# 定义具有相同结构的训练集和验证集

train_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64)
)
val_dataset = tf.data.Dataset.range(50)

# 可重新初始化迭代器根据数据集的结构来定义，我们可以使用训练集也可使用验证集
# 因它们都是相兼容的
iterator = tf.data.Iterator.from_structure(
    tf.data.get_output_types(train_dataset),
    tf.data.get_output_shapes(train_dataset))

next_element = iterator.get_next()

train_init = iterator.make_initializer(train_dataset)
val_init = iterator.make_initializer(val_dataset)

# 20 个 Epochs, 当训练集遍历后，遍历验证集
with tf.Session() as sess:   
    for _ in range(20):
        # 初始化训练集
        sess.run(train_init)
        for _ in range(100):
            sess.run(next_element)
        
        # 初始化验证集，重要的是我们还可以使用同一个 next_element
        sess.run(val_init)
        for _ in range(50):
            sess.run(next_element)


#### 4. 可馈送

可馈送迭代器可以与 `tf.placeholder` 一起使用。它提供的功能**与可重新初始化迭代器相同**，但在迭代器之间切换时**不需要从数据集的开头**初始化迭代器。

例如，上面的同一训练和验证数据集为例，您可以使用 `tf.data.Iterator.from_string_handle` 定义一个可让您**在两个数据集之间切换**的可馈送迭代器

In [40]:
# 定义具有相同结构的训练集和验证集

train_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64)
)
val_dataset = tf.data.Dataset.range(50)

# 我们可以使用训练集也可使用验证集, 因它们都是相兼容的。
# 可馈送通过一个 handle placeholder 和 数据集的结构来定义。
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle,
                                                tf.data.get_output_types(train_dataset),
                                                tf.data.get_output_shapes(train_dataset))
next_element = iterator.get_next()

# 更重要的是我们可以使用不同的 iterator 进行 feed
train_iterator = tf.data.make_one_shot_iterator(train_dataset)
# val_iterator = val_dataset.make_initializable_iterator()
val_iterator = tf.data.make_one_shot_iterator(val_dataset)

with tf.Session() as sess:
    # `Iterator.string_handle()` 返回一个可以馈送的 handle
    train_handle = sess.run(train_iterator.string_handle())
    val_handle = sess.run(val_iterator.string_handle())
    
    # Epochs 20 次
    for _ in range(1):
        # 运行 200 次训练集，是循环迭代, 200 会出错。。。。。迭代完了，好像没有生效
        for _ in range(100):
            sess.run(next_element, feed_dict={handle: train_handle})
        
        for _ in range(50):
            sess.run(next_element, feed_dict={handle: val_handle})
